In [1]:
import pickle
import pandas as pd
from datetime import timedelta
import numpy as np

In [108]:
year=2050
nuc="nuclear_plus"
type_result="multinode_"#"" if singlenode
bati_hyp="ref"
reindus="reindus"
with open("Results_"+type_result+str(year)+"_"+nuc+"_"+bati_hyp+"_"+reindus+".pickle", 'rb') as f:
    Variables=pickle.load(f)

In [109]:
print(Variables)

{'power_Dvar':        AREAS                TIME    TECHNOLOGIES   power_Dvar
0         GB 2019-02-25 07:00:00         OldNuke  1188.000000
1         GB 2019-02-25 07:00:00    WindOffShore  3718.938825
2         GB 2019-02-25 07:00:00         Biomass  4528.000000
3         GB 2019-02-25 07:00:00     WindOnShore  4899.552606
4         GB 2019-02-25 07:00:00             CCG     0.000000
...      ...                 ...             ...          ...
858475    CH 2019-05-26 10:00:00            Coal     0.000000
858476    CH 2019-05-26 10:00:00             TAC     0.000000
858477    CH 2019-05-26 10:00:00      Geothermal     0.000000
858478    CH 2019-05-26 10:00:00  HydroReservoir     0.000000
858479    CH 2019-05-26 10:00:00         NewNuke     0.000000

[858480 rows x 4 columns], 'exchange_Pvar':        AREAS AREAS                TIME  exchange_Pvar
0         GB    GB 2019-02-25 07:00:00            0.0
1         GB    GB 2019-05-17 00:00:00            0.0
2         GB    GB 2019-02-10 08:0

# 1. Capacity 

In [71]:
# Production means capacity (MW)
Variables['capacity_Dvar'][Variables['capacity_Dvar'].AREAS=='FR']

,AREAS,TECHNOLOGIES,capacity_Dvar
70,FR,OldNuke,13870.0000
71,FR,WindOffShore,49000.0000
72,FR,Biomass,3000.0000
73,FR,WindOnShore,37000.0000
74,FR,CCG,0.0000
75,FR,NewHydroRiver,1200.0000
76,FR,Solar,152960.2812
77,FR,curtailment,0.0000
78,FR,HydroRiver,11030.0000
79,FR,Coal,0.0000


In [72]:
# Storage capacity
Variables['Pmax_in_Dvar']

,AREAS,STOCK_TECHNO,Pmax_in_Dvar
0,GB,HydroStorage,6844.422849
1,GB,Battery,9973.546566
2,GB,PowertoH2toPower,15874.290159
3,DE,HydroStorage,7400.000000
4,DE,Battery,19730.092733
5,DE,PowertoH2toPower,63119.345051
6,IT,HydroStorage,19126.752972
7,IT,Battery,7232.728975
8,IT,PowertoH2toPower,20301.715613
9,BE,HydroStorage,1308.000000


In [73]:
# Storage capacity
Variables['Pmax_out_Dvar']

,AREAS,STOCK_TECHNO,Pmax_out_Dvar
0,GB,HydroStorage,6844.422849
1,GB,Battery,9973.546566
2,GB,PowertoH2toPower,6059.042004
3,DE,HydroStorage,7400.000000
4,DE,Battery,19730.092733
5,DE,PowertoH2toPower,45487.431809
6,IT,HydroStorage,19126.752972
7,IT,Battery,7232.728975
8,IT,PowertoH2toPower,6277.965246
9,BE,HydroStorage,1308.000000


In [74]:
# Storage capacity
Variables['Cmax_Pvar']

,AREAS,STOCK_TECHNO,Cmax_Pvar
0,GB,HydroStorage,8.007975e+04
1,GB,Battery,1.049847e+04
2,GB,PowertoH2toPower,2.579361e+06
3,DE,HydroStorage,3.996000e+04
4,DE,Battery,2.411701e+04
5,DE,PowertoH2toPower,2.119882e+07
6,IT,HydroStorage,2.237830e+05
7,IT,Battery,7.613399e+03
8,IT,PowertoH2toPower,8.244946e+06
9,BE,HydroStorage,4.708800e+03


In [75]:
Variables['total_consumption_Pvar']['total_consumption_Pvar'].max()

148910.4360652248

In [100]:
Pow_df=Variables['power_Dvar']
Pow_df=Pow_df[Pow_df.AREAS=='FR']
Pow_df['power_Dvar'].sum()/1E6

805.9462186422799

In [101]:
Conso_df=Variables['total_consumption_Pvar']
Conso_df=Conso_df[Conso_df.AREAS=='FR']
Conso_df['total_consumption_Pvar'].sum()/1E6

682.7512848930962

In [102]:
P2H2_df=Variables['storageIn_Pvar']
P2H2_df=P2H2_df[(P2H2_df.STOCK_TECHNO=='PowertoH2toPower')&(P2H2_df.AREAS=='FR')]
P2H2_df['storageIn_Pvar'].sum()/1E6

96.23518718954679

In [103]:
H2P_df=Variables['storageOut_Pvar']
H2P_df=H2P_df[(H2P_df.STOCK_TECHNO=='PowertoH2toPower')&(P2H2_df.AREAS=='FR')]
H2P_df['storageOut_Pvar'].sum()/1E6

0.0

In [80]:
2.399352e+04/(9212.356899*0.95)

2.741571686703414

In [73]:
# Storage means capacity (MW)
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar          138364.610149
dtype: object

In [74]:
Variables['energy_Pvar'].max()

TIME           2019-12-31 23:00:00
energy_Pvar          138364.610149
dtype: object

In [7]:
df=Variables['power_Dvar']
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')]['power_Dvar'].sum())
df2=Variables['storageOut_Pvar']
print(df2[df2.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df3=Variables['storageIn_Pvar']
print(df3[df3.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df4=Variables['stockLevel_Pvar']
print(df4[df4.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(Variables['stockLevel_ini_Dvar'])

                     TIME    TECHNOLOGIES    power_Dvar
40989 2019-12-31 23:00:00     WindOnShore   4432.600000
40990 2019-12-31 23:00:00             CCG      0.000000
40991 2019-12-31 23:00:00         OldNuke  15520.000000
40992 2019-12-31 23:00:00             TAC      0.000000
40993 2019-12-31 23:00:00    WindOffShore  10598.700000
40994 2019-12-31 23:00:00  HydroReservoir   8785.000000
40995 2019-12-31 23:00:00         Biomass   3000.000000
40996 2019-12-31 23:00:00            Coal      0.000000
40997 2019-12-31 23:00:00           Solar      0.000000
40998 2019-12-31 23:00:00         NewNuke  21710.000000
40999 2019-12-31 23:00:00      HydroRiver   8233.891861
41000 2019-12-31 23:00:00   NewHydroRiver    895.799659
41001 2019-12-31 23:00:00     curtailment      0.000000
73175.99151963575
                    TIME      STOCK_TECHNO  storageOut_Pvar
9459 2019-12-31 23:00:00           Battery              0.0
9460 2019-12-31 23:00:00  PowertoH2toPower              0.0
9461 2019-12-31 23

In [8]:
df=Variables['power_Dvar']
df2=Variables['storageOut_Pvar']
df3=Variables['storageIn_Pvar']
df=df.groupby('TIME')['power_Dvar'].sum()
df2=df2.groupby('TIME')['storageOut_Pvar'].sum()
df3=df3.groupby('TIME')['storageIn_Pvar'].sum()
df_prod=pd.DataFrame()
df_prod['Prod']=df+df2-df3
#df_prod
#df_prod=df_prod.set_index('TIME')

df_diff=pd.DataFrame()
df_diff['diff']=df_prod['Prod']-Variables['total_consumption_Pvar'].set_index('TIME')['total_consumption_Pvar']
print(df_diff['diff'].max())
N_err=0
for i in df_diff.index:
    if np.abs(df_diff.loc[i,'diff'])>1:
        N_err+=1
        print(df_diff.loc[i,'diff'])
print(N_err)

4.3655745685100555e-11
0


In [9]:
df_prod.loc[pd.Timestamp('2019-12-31 23:00:00')]

Prod    73175.99152
Name: 2019-12-31 23:00:00, dtype: float64

In [10]:
Variables['total_consumption_Pvar'].set_index('TIME').loc[pd.Timestamp('2019-12-31 23:00:00')]

total_consumption_Pvar    73175.99152
Name: 2019-12-31 23:00:00, dtype: float64

In [21]:
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar            155289.7205
dtype: object

In [12]:
Variables['total_consumption_Pvar'][Variables['total_consumption_Pvar'].total_consumption_Pvar==Variables['total_consumption_Pvar']['total_consumption_Pvar'].max()]

,TIME,total_consumption_Pvar
1290,2019-01-24 13:00:00,150448.942674


In [13]:
d0=pd.Timestamp('2019-01-01 00:00:00')
d1h=timedelta(hours=1)
df=Variables['flex_consumption_Pvar']
L_EV=[]
for h in range(24):
    L_EV.append(float(df[(df.TIME==d0+h*d1h)&(df.FLEX_CONSUM=='EV')]['flex_consumption_Pvar']))
L_EV

[8974.985765378075,
 11252.569669328535,
 10691.645219955992,
 11170.426644899537,
 13510.099229131103,
 15473.200406998334,
 16550.885092623164,
 4425.99954355268,
 5581.337750877695,
 20919.870411678494,
 21837.125896171616,
 23312.874833511825,
 28469.2682218418,
 25998.15287218346,
 27305.36161875611,
 25600.720420410304,
 20896.11392020079,
 18139.698630793904,
 17262.300474917676,
 15669.700657177991,
 15415.259850726845,
 17359.40017950799,
 25017.329324809274,
 23867.510910719506]

In [110]:
def Total_cost(Variables,area):
    return 1/1e9*(Variables['powerCosts_Pvar'][Variables['powerCosts_Pvar'].AREAS==area]['powerCosts_Pvar'].sum()\
            +Variables['capacityCosts_Pvar'][Variables['capacityCosts_Pvar'].AREAS==area]['capacityCosts_Pvar'].sum()\
            +Variables['storageCosts_Pvar'][Variables['storageCosts_Pvar'].AREAS==area]['storageCosts_Pvar'].sum()\
            +Variables['flexCosts_Pvar'][Variables['flexCosts_Pvar'].AREAS==area]['flexCosts_Pvar'].sum())
Total_cost(Variables,'FR')

39.63335539946913